In [ ]:
import glob
import json
import os
import pickle
import random
import sys
from copy import deepcopy
from typing import Literal, Optional, Union
from pathlib import Path
import matplotlib
import matplotlib.patches as patches
import matplotlib.pyplot as plt
import ml_collections
import numpy as np
import torch
from torch.utils.data import Dataset

from careamics.config.likelihood_model import NMLikelihoodConfig
from careamics.config.nm_model import MultiChannelNMConfig
from careamics.config import VAEAlgorithmConfig
from careamics.lightning import VAEModule
from careamics.lvae_training.eval_utils import (
    Calibration,
    get_calibrated_factor_for_stdev,
    get_dset_predictions,
    get_eval_output_dir,
    plot_calibration,
    plot_error,
    show_for_one,
    stitch_predictions,
)
from careamics.config.architectures import LVAEModel
from careamics.config.nm_model import GaussianMixtureNMConfig, MultiChannelNMConfig
from careamics.config.likelihood_model import (
    GaussianLikelihoodConfig,
    NMLikelihoodConfig,
)
from careamics.config.loss_model import LVAELossConfig, KLLossConfig
from careamics.lvae_training.dataset import (
    LCMultiChDloader,
    MultiChDloader,
    MultiFileDset,
)
from careamics.lvae_training.eval_utils import stitch_predictions_new
from careamics.config.optimizer_models import LrSchedulerModel, OptimizerModel
from careamics.models.lvae.noise_models import noise_model_factory
from careamics.utils.metrics import (
    # avg_psnr,
    # avg_range_inv_psnr,
    # avg_ssim,
    scale_invariant_psnr,
    # multiscale_ssim
)
# TODO: sorry for this :(
sys.path.insert(0, "/home/federico.carrara/Documents/projects/microSplit-reproducibility/")
sys.path.insert(0, "/home/igor.zubarev/projects/microSplit-reproducibility/")

from data import (
  DataSplitType, DataType
)
from data.data_utils import GridAlignement, load_tiff

from datasets import load_train_val_data_nikola, create_train_val_datasets
from configs.nikolaData import get_data_configs

torch.multiprocessing.set_sharing_strategy('file_system')